In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-126407") # UPDATE THIS FOR NEW WORKSPACE
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-126407
Azure region: southcentralus
Subscription id: da775cb9-9ca6-4943-ad21-26dfa99526fc
Resource group: aml-quickstarts-126407


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cpu_cluster_name = "udacity-ml-comp1" # NEEDS TO BE UNIQUE!

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cluster")
except ComputeTargetException:    
    compute_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_V2", 
                                                           max_nodes=4,
                                                          idle_seconds_before_scaledown=1200)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({"--C": uniform(1,5),
                             "--max_iter": choice(10,50,100) }) ### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory=".",entry_script="train.py", compute_target=cpu_cluster) ### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est, hyperparameter_sampling=ps, policy=policy,
                                    primary_metric_name="Accuracy", primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                     max_total_runs="4",
                                    max_concurrent_runs=4) ### YOUR CODE HERE ###

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config) ### YOUR CODE HERE ###
RunDetails(hyperdrive_run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [5]:
import joblib

# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

#Get list of associated filesnames.
print(best_run.get_file_names())

# Download the best run model
best_run.download_file('outputs/model.joblib','outputs/mymodel.joblib')

# Register best model to Azure ML
model = best_run.register_model(model_name='sklearn-banking',model_path='outputs/model.joblib')
### YOUR CODE HERE ###

['--C', '2.582948546153568', '--max_iter', '50']
['azureml-logs/55_azureml-execution-tvmps_736748eeb68ed92541453b9b1ae86bfb153ad1544b52f7420e85a9039dbb8711_d.txt', 'azureml-logs/65_job_prep-tvmps_736748eeb68ed92541453b9b1ae86bfb153ad1544b52f7420e85a9039dbb8711_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_736748eeb68ed92541453b9b1ae86bfb153ad1544b52f7420e85a9039dbb8711_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/103_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']


In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv") ### YOUR CODE HERE ###

### YOUR CODE HERE ###

In [7]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds) ### YOUR DATA OBJECT HERE ###)

# Split into train and test sets.
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

# Concatenate the training features and labels.
import pandas as pd
training_data = pd.concat([x_train,y_train],axis=1)
training_data.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
24024,59,1,0,0,0,7,2,202,2,999,...,0,1,0,0,0,0,0,0,0,0
30643,54,1,0,1,0,8,3,261,1,999,...,0,1,0,0,0,0,0,0,0,0
13651,60,1,0,1,0,7,2,122,13,999,...,0,0,1,0,0,0,0,0,0,0
30037,25,0,0,0,0,5,5,72,2,999,...,1,0,0,0,1,0,0,0,0,0
31557,59,1,0,0,1,5,2,70,1,999,...,1,1,0,0,0,0,0,0,0,0


In [8]:
# Register dataframe as a dataset for inputting into the automl.
from azureml.core import Dataset
target = ws.get_default_datastore()
TabularDatasetFactory.register_pandas_dataframe(training_data,target,name='automl')
dataset = Dataset.get_by_name(ws, name='automl')

WARNING - Method register_pandas_dataframe: This is an experimental method, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/3cc62dfc-06f7-411f-af60-5a77da8c757e/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [9]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=dataset,
    label_column_name='y',
    n_cross_validations=5,
    compute_target = cpu_cluster,
    iterations=20)

In [10]:
# Submit your automl run
exp = Experiment(workspace=ws, name="udacity-project-automl")
run = exp.submit(automl_config, show_output=True)
RunDetails(run).show()
### YOUR CODE HERE ###

Running on remote.
Running on remote compute: udacity-ml-comp1
Parent Run ID: AutoML_b946b4df-5371-4311-92e9-268e9f9d614a

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [11]:
# Retrieve and save your best automl model.
best_run, model = run.get_output()

print(best_run)
print(model)

best_run.register_model(model_name='automl-best',model_path='outputs/model.pkl')
### YOUR CODE HERE ###

Run(Experiment: udacity-project-automl,
Id: AutoML_b946b4df-5371-4311-92e9-268e9f9d614a_18,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                  min_samples_leaf=0.06157894736842105,
                                                                                                  min_samples_split=0.10

Model(workspace=Workspace.create(name='quick-starts-ws-126407', subscription_id='da775cb9-9ca6-4943-ad21-26dfa99526fc', resource_group='aml-quickstarts-126407'), name=automl-best, id=automl-best:1, version=1, tags={}, properties={})

In [12]:
# clean up the computer resouces
cpu_cluster.delete()